# Data predictions

## Imports

In [1]:
import pandas as pd 
import datetime  
from statsmodels.tsa.stattools import adfuller
import json 
from pmdarima import auto_arima
import statsmodels.api as sm
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Import the clean data

In [2]:
data = pd.read_csv('clean-data.csv')

## Set the timestamp as index

In [3]:
data.set_index('timestamp', inplace=True)
data.sort_index(inplace=True)

## Group the data by location

For a more accurate forecasting, I decided to split the dataframe based on the locations. This way we can get a clear understanding of the pollution level for each place in Belgium. This can also help the user to identify the pollution for a specific location.

#### Check how many unique locations are there

In [4]:
data.nunique()

id                 312
latitude           306
longitude          617
pollution        18744
traffic_level    14668
wind_speed          71
pressure           152
temp               112
dtype: int64

There are 306 unique locations.

#### Group the data by location

In [5]:
grouped_data = data.groupby(['latitude', 'longitude'])
 
locations = {}

for (lat, long), group in grouped_data:
    location_key = (lat, long)
    locations[location_key] = group

#### Make a new dataframe for each location

In [6]:
location_dataframes = {}

for location, time_series in locations.items():
    location_df = pd.DataFrame(time_series)
    location_dataframes[location] = location_df

## The dataframes don't have a timestamp

Having a timestamp is important for time series analysis. We will add a timestamp to each dataframe. We will make the timestamps the same way we did them for the initial dataframe, only this time they seem more logical.

In [7]:
current_time = datetime.datetime.now()

for location, df in location_dataframes.items():
    periods = len(df)

    # Calculate the start time for the timestamp column for each dataframe
    start_time = current_time - pd.Timedelta(minutes=5 * (periods - 1))

    # Generate a date range starting from the calculated start time and with the calculated periods
    date_range = pd.date_range(start=start_time, periods=periods, freq='60min')

    # Set the date range as the index and sort by it
    df['timestamp'] = date_range
    df.set_index('timestamp', inplace=True)
    df.sort_index(inplace=True)

This way the dataframes all have the same timestamps, and they are almost ready for the time series analysis.

#### Convert timestamps to datetime

In [8]:
for location, df in location_dataframes.items(): 
    if 'timestamp' in df.columns:
        df.set_index('timestamp', inplace=True)
        
    df.index = pd.to_datetime(df.index, format="%Y-%m-%d %H:%M:%S.%f")

## Save the last info from each dataframe and save them as a JSON file

Save the grouped data into a JSON file to send it to the frontend. I retrieved the last entry from each location because there are too many and most likely not that relevant to the user.

In [9]:
envDataDict = []
 
for location, df in location_dataframes.items():
    # Get the last row in the DataFrame, which corresponds to the latest timestamp
    last_row = df.iloc[-1]
 
    timestamp = last_row.name.strftime('%Y-%m-%d %H:%M:%S')  
    data_id = last_row['id']
    latitude = location[0]
    longitude = location[1]
    pollution = last_row['pollution']
    traffic_level = last_row['traffic_level']
    wind_speed = last_row['wind_speed']
    pressure = last_row['pressure']
    temp = last_row['temp'] - 273.15  # Convert from Kelvin to Celsius
 
    envData = {
        "id": data_id,
        "timestamp": timestamp,
        "latitude": latitude,
        "longitude": longitude,
        "pollution": pollution,
        "traffic_level": traffic_level,
        "wind_speed": wind_speed,
        "pressure": pressure,
        "temp": temp
    }
 
    envDataDict.append(envData)
 
with open('clean-data.json', 'w') as file: 
    json.dump(envDataDict, file, indent=4)

## Check for enough data

To make sure that we have an accurate prediction we need a substential amount of data, so we'll remove dfs that have very little rows.

In [10]:
for df_number, (location, df) in enumerate(location_dataframes.items(), start=1):
    print(f"Number of rows in DataFrame {df_number} ({location}): {len(df)} \n")

Number of rows in DataFrame 1 ((51.0018474024446, 4.13046599598092)): 85 

Number of rows in DataFrame 2 ((51.0028206024438, 4.12623419598206)): 85 

Number of rows in DataFrame 3 ((51.0028996025198, 4.80384249581454)): 116 

Number of rows in DataFrame 4 ((51.0055025024507, 4.19521649596543)): 116 

Number of rows in DataFrame 5 ((51.007370933543, 4.47852256365214)): 87 

Number of rows in DataFrame 6 ((51.0080093025237, 4.85136869580322)): 47 

Number of rows in DataFrame 7 ((51.0147723024784, 4.46811169589919)): 116 

Number of rows in DataFrame 8 ((51.0172213025149, 4.79677389581782)): 85 

Number of rows in DataFrame 9 ((51.0174961024779, 4.47058309589886)): 1 

Number of rows in DataFrame 10 ((51.0195647025133, 4.78902339582001)): 116 

Number of rows in DataFrame 11 ((51.0200067025125, 4.78303809582154)): 16 

Number of rows in DataFrame 12 ((51.0202599024522, 4.24701769595426)): 85 

Number of rows in DataFrame 13 ((51.028696702532, 4.97625569577416)): 116 

Number of rows in D

## Remove dataframes that don't have enough data for predictions

In [11]:
filtered_dataframes = {} 

for location, df in location_dataframes.items():
    if len(df) >= 100:
        filtered_dataframes[location] = df
        print(len(df))

116
116
116
116
116
116
116
232
116
115
116
116
116
116
116
116
116
116
116
116
108
116
116
116
116
116
116
116
115
116
116
116
116
116
116
116
116
116
116
116
116
118
114
116
116
115
100
103
117
116
116
116
112
116
180
116
101
101
116
115
116
118
116
116
118
118
116
116
116
111
116
116
110
232
101
112
115
116
110
116
116
116
116
114
115
116
116
116
116
116
116
116
116
116
116
116
116
116
102
116
115
116
116
116
116
115
115
116
116
116
115
116
115
112
115
116
116
116
116
116
116
116
103
116
116
115
116
116
115
116
116
116
116
116
116
118
116
116
116
113
116
118
115
116
116
116
116
116
116
104
116
107
116
116
116
116
116
116
116
116
115
116
116
115
110
116
115
116
110
116
116
115
116
116
116
115
111
116
115
113
116
116
116
116
116
116
116
116
116
115
115
115
116


In [12]:
len(filtered_dataframes)

193

## Forecast using SARIMAX

The SARIMAX model (Seasonal AutoRegressive Integrated Moving Average with eXogenous factors), is a statistical method used for forecasting times series data. It is an extension of the ARIMA model and is designed to capture complex patterns in time series data.

1. *AR (AutoRegressive)* captures the relationship between an observation and a number of previously. It assumes that the current value of the series can be explained by its previous values.

2. *MA (Moving Average)* models the relationship between an observation and a residual error from a moving average model applied to lagged observations. It helps to smooth out short-term fluctuations and highlight longer-term trends or cycles.

3. *I (Integrated)* subtracts an observation from an observation at a previous time step so that its statistical properties like mean and variance do not change over time.

4. *Seasonal Components* capture seasonality or periodic patterns in the data.

5. *eXogenous Factors* refers to the inclusion of external variables that might affect the time series but are not part of it. These could be variables like weather conditions, economic indicators, etc., that are believed to have an impact on the variable being forecasted.

The SARIMAX model is particularly useful in scenarios where the data shows both non-seasonal and seasonal patterns and where external factors are expected to influence the series. It provides a comprehensive framework for modeling complex behaviors in time series data, making it a popular choice for forecasting in various fields like economics, finance, environmental science, and traffic analysis.

## Reasons for choosing the SARIMAX model for our predictions:
*   The data is a times series, it being the primary requirement for SARIMAX.
*   The dataset contains external factors that can influence the pollution (traffic, location etc.) SARIMAX incorporates these as exogenous factors, potentially improving the forecast by accounting for their impact on the patterns.
*   SARIMAX is a flexible model that can handle complex data relationships, including non-stationary data, which is common in real-world time series like pollution data.

## How is the SARIMAX model going to help the user

The output from the SARIMAX model provides users with accurate forecasts of future values based on historical data and external factors. This can be particularly useful for decision-making, planning, and optimizing resources based on expected future trends.

## Check for stationarity

Stationarity is an important characteristic of time series data. A stationary time series is one whose statistical properties like mean, variance, autocorrelation, etc., are all constant over time. This means that the mean and variance of the series should not be a function of time. If the series is stationary, it is easier to model its behavior.

In [13]:
def adfuller_test(pollution):
    result = adfuller(pollution) 
    if result[1] <= 0.05:
        print('strong evidence against the null hypothesis(Ho), reject the null hypothesis, data is stationary')
    else:
        print('weak evidence against the null hypothesis(Ho), data is not stationary')

adfuller_test(data['pollution'])

strong evidence against the null hypothesis(Ho), reject the null hypothesis, data is stationary


#### Initialize the dictionaries that will be used for training and predicting the model

In [14]:
sarimax_models = {}
forecasts = {}
forecast_indices = {}
train_test_split = {}
metrics = {}

## Calculate the p, d, q values

- *p*: the number of lag observations in the model, also known as the lag order.
It represents the number of days you want to look back to predict the data
for the next day.

- *d*: the number of times the raw observations are differenced; also known as
the degree of differencing. It is used to see a clearer pattern in your data.
You might look at the changes from one day to the next instead of the actual
values.

- *q*: the size of the moving average window, also known as the order of the
moving average. It is like taking an average of past errors (differences
between actual and predicted values) to smooth out your predictions. If q is
2, you're using the average of the errors from the last two days to help make
today's prediction more accurate.

In [15]:
stepwise_fit = auto_arima(data['pollution'], trace=True)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=329530.812, Time=2.57 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=329523.452, Time=0.29 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=329525.023, Time=0.38 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=329525.051, Time=0.91 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=334611.655, Time=0.17 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=329527.026, Time=1.33 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0] intercept
Total fit time: 5.670 seconds


## Train and predict the model for each location

Because we are iterating over all the dataframes, we will perform the whole training, evaluation and prediction process in a single code cell.

In [16]:
for location, df in filtered_dataframes.items(): 
    ######### Split the data into training and validation sets #########
    split_index = int(0.70 * len(df))
    train_exog = df[['traffic_level', 'pressure']][:split_index]
    train_endog = df['pollution'][:split_index]
    exog = df[['traffic_level', 'pressure']]  

    ######### Train the model for each dataframe #########
    model = sm.tsa.statespace.SARIMAX(df['pollution'], order=(0, 1, 1), seasonal_order=(0, 1, 1, 48))
    results = model.fit()

    sarimax_models[location] = results

    ######## Forecasting #########
    forecast_exog = exog[-48:]  # Use the last 48 hours of exogenous variables 
    forecast = results.forecast(steps=12, exog=forecast_exog)
    forecasts[location] = forecast
 
    last_date = df.index[-1]
    forecast_index = pd.date_range(start=last_date, periods=12, freq='h')
    forecast_indices[location] = forecast_index

c:\Users\bogda\Desktop\integration_retake\integration-5-resit\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
c:\Users\bogda\Desktop\integration_retake\integration-5-resit\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
c:\Users\bogda\Desktop\integration_retake\integration-5-resit\.venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\bogda\Desktop\integration_retake\integration-5-resit\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was

## Plot the predictions 

In [18]:
fig = make_subplots(rows=3, cols=2, subplot_titles=[f"Location: {location}" for location in forecasts.keys()][:6])

for i, (location, forecast) in enumerate(forecasts.items(), 1):
    if i > 6:
        break

    df = filtered_dataframes[location]
    split_index = int(0.75 * len(df))
    actual = df['pollution'][:split_index]
    predicted = sarimax_models[location].fittedvalues if sarimax_models[location] else pd.Series()

    row = (i - 1) // 2 + 1
    col = (i - 1) % 2 + 1
 
    fig.add_trace(
        go.Scatter(x=actual.index, y=actual, mode='lines', name='Actual', line=dict(color='blue')),
        row=row, col=col
    )
 
    fig.add_trace(
        go.Scatter(x=predicted.index, y=predicted, mode='lines', name='Predicted', line=dict(color='red')),
        row=row, col=col
    ) 

fig.update_layout(height=700, width=1200, title_text="Pollution Forecast vs Actual", showlegend=False)
fig.show()

## Save the predicted values as a JSON file 

In [19]:
json_forecasts = []
id_counter = 9000006280 # Last id in the database  

for (lat, lon), forecast_series in forecasts.items():
    for timestamp, pollution_value in forecast_series.items(): 
        forecast_entry = {
            "id": id_counter,  
            "timestamp": timestamp.strftime("%Y-%m-%d %H:%M:%S"),
            "latitude": lat,
            "longitude": lon,
            "pollution": pollution_value, 
        }
        json_forecasts.append(forecast_entry)
        id_counter += 1

with open('forecasts.json', 'w') as file:
    json.dump(json_forecasts, file, indent=4) 


## Integration

After creating the JSON files, the data will be transfered using APIs to the React frontend.